## 208. Implement Trie (Prefix Tree)

### self-written solution after checking answers

In [1]:
class TrieNode:
    
    charSize = 26
    
    def __init__(self):
        self.links = [None] * TrieNode.charSize
        self.isEnd_ = False

    def containsKey(self, char):
        return self.links[ord(char) - ord('a')] is not None
    
    def get(self, char):
        return self.links[ord(char) -ord('a')]
    
    def put(self, char, node):
        self.links[ord(char) - ord('a')] = node
    
    def setEnd(self):
        self.isEnd_ = True
    
    def isEnd(self):
        return self.isEnd_

    
class Trie:

    def __init__(self):
        """
        Initialize your data structure here.
        """
        self.root = TrieNode()

    def insert(self, word: str) -> None:
        """
        Inserts a word into the trie.
        """
        node = self.root
        for char in word:
            if not node.containsKey(char):
                node.put(char, TrieNode())
            node = node.get(char)
        node.setEnd()

    def search(self, word: str) -> bool:
        """
        Returns if the word is in the trie.
        """
        node = self.root
        for char in word:
            if node.containsKey(char):
                node = node.get(char)
            else:
                return False
        return node.isEnd()

    def startsWith(self, prefix: str) -> bool:
        """
        Returns if there is any word in the trie that starts with the given prefix.
        """
        node = self.root
        for char in prefix:
            if node.containsKey(char):
                node = node.get(char)
            else:
                return False
        return True

### self-written solution (faster and simpler)

In [2]:
class Trie:

    def __init__(self):
        """
        Initialize your data structure here.
        """
        self.root = {}

    def insert(self, word: str) -> None:
        """
        Inserts a word into the trie.
        """
        node = self.root
        for char in word:
            if char not in node:
                node[char] = {}
            node = node.get(char)
        node['#'] = True # True is not important here actually

    def search(self, word: str) -> bool:
        """
        Returns if the word is in the trie.
        """
        node = self.root
        for char in word:
            if char in node:
                node = node.get(char)
            else:
                return False
        return '#' in node

    def startsWith(self, prefix: str) -> bool:
        """
        Returns if there is any word in the trie that starts with the given prefix.
        """
        node = self.root
        for char in prefix:
            if char in node:
                node = node.get(char)
            else:
                return False
        return True

## 146. LRU (Least Recently Used) Cache

### self-written solution after checking answers

In [33]:
# need two structures: linked list and hashmap
# hashmap linked key to node and linked list perform O(1) eviction and O(1) insertion
# key -> node enables fast insertion and eviction

class DLinkedNode: 
    
    def __init__(self):
        self.key = 0
        self.value = 0
        self.prev = None
        self.next = None

class LRUCache:

    def __init__(self, capacity: int): # important that capacity > 0
        self.cache = {} # key -> node
        self.capacity = capacity
        self.size = 0
        # initialize two dummy nodes for easy eviction and insertion
        self.head, self.tail = DLinkedNode(), DLinkedNode()
        # add linkds between head and tail
        self.head.next = self.tail
        self.tail.prev = self.head
    
    def _remove_node(self, node):
        prev_node = node.prev
        next_node = node.next
        prev_node.next = next_node
        next_node.prev = prev_node
    
    def _add_node(self, node):
        # always add node right after head
        node.next = self.head.next
        self.head.next.prev = node
        
        self.head.next = node
        node.prev = self.head
        
    # eviction
    def _pop_tail(self):
        # need to return key value of the least recently used key to update hashmap
        node_to_be_popped = self.tail.prev
        self._remove_node(node_to_be_popped)
        return node_to_be_popped.key
    
    def _move_to_head(self, node):
        self._remove_node(node)
        self._add_node(node)      

    def get(self, key: int) -> int:
        # can't simply use return -1 if key nnot in self.cache else self.cache[key].value
        # need to update order
        if key not in self.cache:
            return -1  
        else:
            node = self.cache[key]
            self._move_to_head(node)
            return node.value

    def put(self, key: int, value: int) -> None:
        if key in self.cache:
            node = self.cache[key]
            node.value = value
            self._move_to_head(node)
        else:
            node = DLinkedNode()
            node.key = key
            node.value = value
            if self.size == self.capacity:
                removed_key = self._pop_tail()
#                 print(removed_key)
                del self.cache[removed_key]
                self.size -= 1
            self._add_node(node)
            self.cache[key] = node
            self.size += 1
#         print(self.cache.keys())

In [34]:
lRUCache = LRUCache(2)
lRUCache.put(1, 1) # cache is {1=1}
lRUCache.put(2, 2) # cache is {1=1, 2=2}
print(lRUCache.get(1)) # return 1
lRUCache.put(3, 3), # LRU key was 2, evicts key 2, cache is {1=1, 3=3}
print(lRUCache.get(2)) # returns -1 (not found)
lRUCache.put(4, 4) # LRU key was 1, evicts key 1, cache is {4=4, 3=3}
print(lRUCache.get(1)) # return -1 (not found)
print(lRUCache.get(3)) # return 3
print(lRUCache.get(4)) # return 4

1
-1
-1
3
4


## 353. Design Snake Game

### self-written solution

In [48]:
from typing import List, Tuple

In [75]:
from collections import deque

# this implementation is quite easy and straightforward
# hashset to check existence and queue to perform O(1) append and pop
class SnakeGame:

    def __init__(self, width: int, height: int, food: List[List[int]]):
        self.score = 0
        self.curr_pos = (0, 0)
        self.food_index = 0
        self.queue = deque()
        self.hashset = set()
        self.width = width
        self.height = height
        self.food = [(x, y) for x, y in food]
    
    def _get_next_pos(self, direction: str) -> Tuple[int]:
        if direction == "R":
            return (self.curr_pos[0], self.curr_pos[1] + 1)
        elif direction == "L":
            return (self.curr_pos[0], self.curr_pos[1] - 1)
        elif direction == "U":
            return (self.curr_pos[0] - 1, self.curr_pos[1])
        elif direction == "D":
            return (self.curr_pos[0] + 1, self.curr_pos[1])
        return -1, -1
    
    def _is_valid_pos(self, pos: Tuple[int]) -> bool:
        return (0 <= pos[0] <= (self.height - 1)) and \
               (0 <= pos[1] <= (self.width - 1)) and \
               (pos not in self.hashset)
        
    def move(self, direction: str) -> int:
        next_pos = self._get_next_pos(direction)
        is_valid = self._is_valid_pos(next_pos)
        if is_valid:
            is_food = self.food_index < len(self.food) and \
                      self.food[self.food_index] == next_pos
            self.queue.append(next_pos)
            self.hashset.add(next_pos)
            self.curr_pos = next_pos
            if is_food:
                self.score += 1
                self.food_index += 1
            else:
                pos_to_be_removed = self.queue.popleft()
                self.hashset.remove(pos_to_be_removed)
            return self.score
        else:
            return -1

In [76]:
snakeGame = SnakeGame(3, 2, [[1, 2], [0, 1]])
print(snakeGame.move("R")) # return 0
print(snakeGame.move("D")) # return 0
print(snakeGame.move("R")) 
# return 1, snake eats the first piece of food. The second piece of food appears at (0, 1).
print(snakeGame.move("U")) # return 1
print(snakeGame.move("L")) # return 2, snake eats the second food. No more food appears.
print(snakeGame.move("U")) # return -1, game over because snake collides with border

0
0
1
1
2
-1


## 155. Min Stack

### self-written solution after checking answers

#### the trick for this question is the "invariant": the numbers below the top of the stack never change, so do their minimum value

In [83]:
class MinStack:

    def __init__(self):
        self.stack = deque()      

    def push(self, val: int) -> None:
        min_val = min(self.stack[-1][1], val) if self.stack else val
        self.stack.append((val, min_val))   

    def pop(self) -> None:
        self.stack.pop()

    def top(self) -> int:
        return self.stack[-1][0]   

    def getMin(self) -> int:
        return self.stack[-1][1]

In [84]:
minStack = MinStack()
minStack.push(-2)
minStack.push(0)
minStack.push(-3)
print(minStack.getMin()) # return -3
minStack.pop()
print(minStack.top()) # return 0
print(minStack.getMin()) # return -2

-3
0
-2


#### implementation with two stacks for less memory usage

In [89]:
class MinStack:

    def __init__(self):
        self.stack = deque()
        self.min_tracker = deque()

    def push(self, val: int) -> None:
        curr_min = self.min_tracker[-1] if self.min_tracker else val
        if curr_min >= val:
            self.min_tracker.append(val)
        self.stack.append(val)            

    def pop(self) -> None:
        if self.min_tracker[-1] == self.stack[-1]:
            self.min_tracker.pop()
        self.stack.pop()

    def top(self) -> int:
        return self.stack[-1]  

    def getMin(self) -> int:
        return self.min_tracker[-1]

In [90]:
minStack = MinStack()
minStack.push(-2)
minStack.push(0)
minStack.push(-3)
print(minStack.getMin()) # return -3
minStack.pop()
print(minStack.top()) # return 0
print(minStack.getMin()) # return -2

-3
0
-2


#### implementation with two stacks for maybe even less memory usage

In [91]:
class MinStack:

    def __init__(self):
        self.stack = deque()
        self.min_tracker = deque()

    def push(self, val: int) -> None:
        if not self.min_tracker:
            self.min_tracker.append([val, 0])
        curr_min = self.min_tracker[-1][0]
        if curr_min > val:
            self.min_tracker.append([val, 1])
        elif curr_min == val:
            self.min_tracker[-1][1] += 1
        self.stack.append(val)            

    def pop(self) -> None:
        if self.min_tracker[-1][0] == self.stack[-1]:
            self.min_tracker[-1][1] -= 1
            if self.min_tracker[-1][1] == 0:
                self.min_tracker.pop()
        self.stack.pop()

    def top(self) -> int:
        return self.stack[-1]  

    def getMin(self) -> int:
        return self.min_tracker[-1][0]

In [92]:
minStack = MinStack()
minStack.push(-2)
minStack.push(0)
minStack.push(-3)
print(minStack.getMin()) # return -3
minStack.pop()
print(minStack.top()) # return 0
print(minStack.getMin()) # return -2

-3
0
-2


## 1146. Snapshot Array

### self-written solution, time limit exceeded

In [103]:
"""
this solution is brutal, with more memory used but faster when retrieving results
another alternative is to use an list of tuples to store snapshots that changed the value of each entry
that design stores less value and speeds up the setting process if adjacent snapshots differ little but 
requires binary search to return a certain snapshot
"""
class SnapshotArray:

    def __init__(self, length: int):
        self.length = length
        self.array = [0] * length
        self.history = [[] for _ in range(length)]
        self.count = 0

    def set(self, index: int, val: int) -> None:
        self.array[index] = val

    def snap(self) -> int:
        for i in range(self.length):
            self.history[i].append(self.array[i])
        self.count += 1
        return self.count - 1        

    def get(self, index: int, snap_id: int) -> int:
        return self.history[index][snap_id]

In [104]:
snapshotArr = SnapshotArray(3) # set the length to be 3
snapshotArr.set(0,5) # Set array[0] = 5
print(snapshotArr.snap()) # Take a snapshot, return snap_id = 0
snapshotArr.set(0,6)
print(snapshotArr.get(0,0)) # Get the value of array[0] with snap_id = 0, return 5

0
5


### self-written solution with binanry search

In [ ]:
class SnapshotArray:

    def __init__(self, length: int):
        self.length = length
        self.array = [0] * length
        self.history = [[] for _ in range(length)]
        self.count = 0

    def set(self, index: int, val: int) -> None:
        self.array[index] = val

    def snap(self) -> int:
        for i in range(self.length):
            snapshots = self.history[i]
            value = self.array[i]
            # snap_id would be 0 for the first case
            if not snapshots or snapshots[-1][1] != value:
                snapshots.append((self.count, self.array[i]))
        self.count += 1
        return self.count - 1        

    def get(self, index: int, snap_id: int) -> int:
        